In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

%matplotlib inline

Scrape the Wikipedia page, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe .

In [21]:
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html_doc = req.text
soup = BeautifulSoup(html_doc,'lxml')
#print(soup)

In [22]:
table_show = soup.find('table',class_='wikitable sortable')
table=[]
for item in table_show.find_all('tr'):
    data_columns = item.text
    rows = data_columns.strip().split('\n')
    table.append(rows)


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [24]:
column_name = table[0]
table_content = table[1:len(table)]
post_code_gt = pd.DataFrame(data=table_content, columns=column_name)
post_code_gt = post_code_gt[post_code_gt.Borough != 'Not assigned']
post_code_gt.index = range(len(post_code_gt))
post_code_gt.head(10)

,Postal Code,,Borough,,Neighbourhood
0,M3A,,North York,,Parkwoods
1,M4A,,North York,,Victoria Village
2,M5A,,Downtown Toronto,,"Regent Park, Harbourfront"
3,M6A,,North York,,"Lawrence Manor, Lawrence Heights"
4,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government"
5,M9A,,Etobicoke,,"Islington Avenue, Humber Valley Village"
6,M1B,,Scarborough,,"Malvern, Rouge"
7,M3B,,North York,,Don Mills
8,M4B,,East York,,"Parkview Hill, Woodbine Gardens"
9,M5B,,Downtown Toronto,,"Garden District, Ryerson"


Combining Neighbourhoods by Postal Code/Borough.

In [25]:
post_code_gt = post_code_gt.groupby(['Postal Code','Borough']).agg(','.join)
post_code_gt = post_code_gt.reset_index(level=['Postal Code', 'Borough'])
post_code_gt.head(10)

,Postal Code,Borough,,,Neighbourhood
0,M1B,Scarborough,,,"Malvern, Rouge"
1,M1C,Scarborough,,,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,,,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,,,Woburn
4,M1H,Scarborough,,,Cedarbrae
5,M1J,Scarborough,,,Scarborough Village
6,M1K,Scarborough,,,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,,,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,,,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,,,"Birch Cliff, Cliffside West"


Check if there has any Neighbourhoods not assigned.

In [26]:
post_code_gtna = post_code_gt.loc[post_code_gt['Neighbourhood'] == 'Not assigned']
post_code_gtna

,Postal Code,Borough,,,Neighbourhood


Using the .shape method to print the number of rows of this dataframe.

In [27]:
print(post_code_gt.shape)

(103, 5)
